## Projeto

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane.optimize import AdamOptimizer
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AngleEmbedding
from tqdm.notebook import tqdm, trange
from qiskit_ibm_provider import IBMProvider

In [2]:
# Read the HDF5 file using pandas
data_frame_fcnc = pd.read_hdf('fcnc_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_fcnc.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_fcnc.head()


Number of rows: 90548


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,1.408853,15.150869,339.182312,2.350262,1.396943,0.710451,0.109013,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,test,7.762202e-09,0.001285
1,1,-2.481838,7.208333,247.036240,-2.280740,0.428710,0.205213,0.000000,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,val,7.762202e-09,0.001285
2,0,1.476267,94.220718,238.014694,-1.788097,94.256210,2.418446,1.585315,1.127324,0.431098,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
3,1,0.647480,13.459283,230.971832,-1.032663,1.227122,0.467150,0.164008,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
4,0,2.106436,97.490242,698.399902,-3.059983,36.555862,2.937936,1.799140,1.093004,0.589724,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285


In [3]:
# Read the HDF5 file using pandas
data_frame_bkg = pd.read_hdf('bkg_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_bkg.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_bkg.head()

Number of rows: 1002490


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,-1.988600,52.710262,229.350952,0.728242,36.148926,23.039709,16.949991,14.424411,12.000529,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,train,0.000018,11.9635
1,0,0.528382,61.115589,315.538910,-0.863614,32.592808,22.366640,16.285843,13.938633,11.180016,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,test,0.000018,11.9635
2,0,1.479911,98.012802,251.109573,-3.133624,90.252274,33.646885,30.612156,27.973904,23.729696,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
3,2,0.926899,-0.000007,240.909348,0.835656,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
4,0,0.781194,72.234299,206.020386,-0.320449,48.886372,20.743645,16.572512,13.070706,11.269534,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635


### VQC

#### Pennylane (V1)

In [4]:
# Split the original data into train, validation and test sets for each dataset
data_frame_fcnc_train = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'train']
data_frame_bkg_train =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'train']

data_frame_fcnc_test = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'test']
data_frame_bkg_test =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'test']

data_frame_fcnc_val = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'val']
data_frame_bkg_val =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'val']


# get 5'' points of each dataset and  join the datasets (randomly)
train_fcnc = data_frame_fcnc_train.sample(n=500)
train_bkg = data_frame_bkg_train.sample(n=500)
train = pd.concat([train_fcnc, train_bkg])
train = train.sample(frac=1)

test_fcnc = data_frame_fcnc_test.sample(n=500)
test_bkg = data_frame_bkg_test.sample(n=500)
test= pd.concat([test_fcnc, test_bkg])
test = test.sample(frac=1)

val_fnc = data_frame_fcnc_val.sample(n=500)
val_bkg = data_frame_bkg_val.sample(n=500)
val = pd.concat([val_fnc, val_bkg])
val = val.sample(frac=1)

# get the weights for each dataset
w_train = train[['gen_xsec']]
w_test = test[['gen_xsec']]
w_val = val[['gen_xsec']]


# change the signal and bkg labels to 0 and 1 and get the labels for each dataset
train = train.replace(['signal'], 1)
train= train.replace(['bkg'], 0)
y_train = train[['gen_label']]
x_train = train[['MissingET_MET', 'Jet1_BTag']]

test = test.replace(['signal'], 1)
test= test.replace(['bkg'], 0)
y_test = test[['gen_label']]
x_test = test[['MissingET_MET', 'Jet1_BTag']]

val = val.replace(['signal'], 1)
val= val.replace(['bkg'], 0)
y_val = val[['gen_label']]
x_val = val[['MissingET_MET', 'Jet1_BTag']]

y_train_arr = np.concatenate(y_train.values, axis=0 )
y_val_arr = np.concatenate( y_val.values, axis=0 )
y_test_arr = np.concatenate( y_test.values, axis=0 )

# Renormalize data weights
w_train[y_train_arr == 1] = (w_train[y_train_arr == 1] / w_train[y_train_arr == 1].sum()) * y_train_arr.shape[0] / 2
w_train[y_train_arr == 0] = (w_train[y_train_arr == 0] / w_train[y_train_arr == 0].sum()) * y_train_arr.shape[0] / 2
        
w_test[y_test_arr == 1] = (w_test[y_test_arr == 1] / w_test[y_test_arr == 1].sum()) * w_test.shape[0] / 2
w_test[y_test_arr == 0] = (w_test[y_test_arr == 0] / w_test[y_test_arr == 0].sum()) * w_test.shape[0] / 2
        
w_val[y_test_arr == 1] = (w_val[y_test_arr == 1] / w_val[y_test_arr == 1].sum()) * w_val.shape[0] / 2
w_val[y_test_arr == 0] = (w_val[y_test_arr == 0] / w_val[y_test_arr == 0].sum()) * w_val.shape[0] / 2

# Concatenate features
X = np.concatenate([x_train, x_test,x_val])

# Normalize the data for angle embeding  (Put the data between -pi and pi)
X = (((X - X.min()) / (X.max() - X.min())) * 2 - 1) * (np.pi)

# Split the features array into train, validation and test sets
x_train = X[:1000]
x_test = X[1000:2000]
x_val = X[2000:]

In [5]:
# accuracy function
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss


In [6]:

# loss function
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

# quantum circuit function
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

# classifier function    
def classifier(n_features, n_layers, weights, x):
        #c = circuit(n_features, n_layers, weights, x)
        dev=qml.device("default.qubit", wires=n_features)
        return qml.QNode(circuit, dev)(n_features, n_layers, weights, x)
    
# cost function    
def cost(n_features, n_layers,weights,X,Y,W):  
        # Compute predictions
        y_scores = [(classifier(n_features, n_layers,weights, x) + 1) / 2 for x in X]

        loss = square_loss(Y, y_scores)
        loss = loss * W
        loss = loss.sum()
        
        return loss
    
# train step function    
def train_step(n_features, n_layers,x_train,y_train, w_train, weights, opt,desc='Training'):
        
        # Only require grad if necessary
        x_train = np.array(x_train, requires_grad=False)
        y_train = np.array(y_train, requires_grad=True)
        w_train = np.array(w_train, requires_grad=False)

        # Compute cost and update weights
        weights, loss = opt.step_and_cost(cost, n_features, n_layers,weights, X=x_train, Y=y_train, W=w_train)

        return loss, weights
    
# validation step function
def validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch,best_weights,desc='Validation'):
    X_val = np.array(x_val, requires_grad=False)
    Y_val = np.array(y_val, requires_grad=False)
    W_val = np.array(w_val, requires_grad=False)

    y_scores = np.array([classifier(n_features, n_layers, weights, x) for x in X_val])
    y_scores = (y_scores + 1) / 2

    W_val[Y_val == 1] = (W_val[Y_val == 1] / W_val[Y_val == 1].sum()) * W_val.shape[0] / 2
    W_val[Y_val == 0] = (W_val[Y_val == 0] / W_val[Y_val == 0].sum()) * W_val.shape[0] / 2

    auc_score = roc_auc_score(y_true=Y_val, y_score=y_scores, sample_weight=W_val)
    loss = cost(n_features, n_layers, weights, X_val, Y_val, W_val)


    if best_score is None or auc_score > best_score:
        best_score = auc_score
        best_score_epoch = epoch_number
        best_weights = weights

    tqdm.write(f"Epoch: {epoch_number}, Validation Loss: {loss:.4f}, AUC Score: {auc_score:.4f}")

    return best_score, best_score_epoch, best_weights
        
        
        
# train function
def train(n_features, n_layers, x_train, y_train, learning_rate, weights, max_epochs, epoch_number):
    opt = AdamOptimizer(learning_rate)
    best_score = None
    best_weights = None
    best_score_epoch = None

    with tqdm(total=max_epochs, desc='Epoch', unit='epoch') as pbar:
        for epoch in range(epoch_number, max_epochs):
            epoch_number = epoch

            loss, nf_nl_weights = train_step(n_features, n_layers, x_train, y_train, w_train, weights, opt, desc='Training')
            
            # Log variable values using tqdm.write
            tqdm.write(f"Epoch: {epoch_number:}, Loss: {loss:.4f}")
            
            
            weights = nf_nl_weights[2:]
            weights = weights[0]

            if epoch_number == max_epochs - 1 or (epoch_number+1)%5==0:
                best_score, best_score_epoch, best_weights = validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch, best_weights,desc='Validation')
                # early stopping
                if epoch_number - best_score_epoch > 30 and epoch_number > 80:
                    tqdm.write(f"Early stopping at epoch {epoch_number}")
                    break

            pbar.update(1)  # Update progress bar
        tqdm.write(f"Best Score: {best_score:.4f}")            
        
    return best_score, best_weights

### Default Device

In [7]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 1000, 0)


Epoch:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 510.0425
Epoch: 1, Loss: 509.8386
Epoch: 2, Loss: 509.2260
Epoch: 3, Loss: 508.1480
Epoch: 4, Loss: 506.5491
Epoch: 4, Validation Loss: 510.0878, AUC Score: 0.2049
Epoch: 5, Loss: 504.3978
Epoch: 6, Loss: 501.6759
Epoch: 7, Loss: 498.3757
Epoch: 8, Loss: 494.4996
Epoch: 9, Loss: 490.0604
Epoch: 9, Validation Loss: 490.7805, AUC Score: 0.2045
Epoch: 10, Loss: 485.0823
Epoch: 11, Loss: 479.6010
Epoch: 12, Loss: 473.6635
Epoch: 13, Loss: 467.3264
Epoch: 14, Loss: 460.6516
Epoch: 14, Validation Loss: 459.4497, AUC Score: 0.2053
Epoch: 15, Loss: 453.6963
Epoch: 16, Loss: 446.5009
Epoch: 17, Loss: 439.0881
Epoch: 18, Loss: 431.4735
Epoch: 19, Loss: 423.6696
Epoch: 19, Validation Loss: 421.4074, AUC Score: 0.2063
Epoch: 20, Loss: 415.6888
Epoch: 21, Loss: 407.5482
Epoch: 22, Loss: 399.2709
Epoch: 23, Loss: 390.8836
Epoch: 24, Loss: 382.4153
Epoch: 24, Validation Loss: 379.1753, AUC Score: 0.2060
Epoch: 25, Loss: 373.8958
Epoch: 26, Loss: 365.3563
Epoch: 27, Loss: 356.8299
Epoc

In [8]:
print (best_score)

0.7890943370681786


In [9]:
# test the model

def test(n_features, n_layers,x_test,y_test,w_test, weights):
        # Remove grad
        X_test = np.array(x_test, requires_grad=False)
        Y_test = np.array(y_test, requires_grad=False)
        W_test = np.array(w_test, requires_grad=False)

        # This will be between -1 and 1, we need to convert to between 0 and 1
        y_scores = np.array([classifier(n_features, n_layers,weights, x) for x in X_test])
        y_scores = (y_scores + 1) / 2

        # Renormalize weights
        W_test[Y_test == 1] = (W_test[Y_test == 1] / W_test[Y_test == 1].sum()) * W_test.shape[0] / 2
        W_test[Y_test == 0] = (W_test[Y_test == 0] / W_test[Y_test == 0].sum()) * W_test.shape[0] / 2

        # Calculate ROC
        auc_score = roc_auc_score(y_true=Y_test, y_score=y_scores, sample_weight=W_test)
        
        return auc_score
    
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7568898558963438


#### default.qubit.tf Device

In [10]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits) 
dev =qml.device('default.qubit.tf', wires=n_features) 
 
# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 1000, 0)

2023-07-12 10:29:57.323912: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 10:29:57.325997: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 10:29:57.362884: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 10:29:57.363899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 10:29:57.836434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 509.9651
Epoch: 1, Loss: 509.3178
Epoch: 2, Loss: 508.1373
Epoch: 3, Loss: 506.3903
Epoch: 4, Loss: 504.0652
Epoch: 4, Validation Loss: 506.8460, AUC Score: 0.2050
Epoch: 5, Loss: 501.1568
Epoch: 6, Loss: 497.6660
Epoch: 7, Loss: 493.6026
Epoch: 8, Loss: 488.9864
Epoch: 9, Loss: 483.8470
Epoch: 9, Validation Loss: 483.9308, AUC Score: 0.2040
Epoch: 10, Loss: 478.2245
Epoch: 11, Loss: 472.1687
Epoch: 12, Loss: 465.7373
Epoch: 13, Loss: 458.9918
Epoch: 14, Loss: 451.9898
Epoch: 14, Validation Loss: 450.5407, AUC Score: 0.2048
Epoch: 15, Loss: 444.7758
Epoch: 16, Loss: 437.3762
Epoch: 17, Loss: 429.7998
Epoch: 18, Loss: 422.0462
Epoch: 19, Loss: 414.1195
Epoch: 19, Validation Loss: 411.7187, AUC Score: 0.2063
Epoch: 20, Loss: 406.0341
Epoch: 21, Loss: 397.8136
Epoch: 22, Loss: 389.4869
Epoch: 23, Loss: 381.0853
Epoch: 24, Loss: 372.6400
Epoch: 24, Validation Loss: 369.3196, AUC Score: 0.2059
Epoch: 25, Loss: 364.1818
Epoch: 26, Loss: 355.7426
Epoch: 27, Loss: 347.3559
Epoc

In [ ]:
print (best_score)

0.7221942085766418


In [ ]:
# test the model

def test(n_features, n_layers,x_test,y_test,w_test, weights):
        # Remove grad
        X_test = np.array(x_test, requires_grad=False)
        Y_test = np.array(y_test, requires_grad=False)
        W_test = np.array(w_test, requires_grad=False)

        # This will be between -1 and 1, we need to convert to between 0 and 1
        y_scores = np.array([classifier(n_features, n_layers,weights, x) for x in X_test])
        y_scores = (y_scores + 1) / 2

        # Renormalize weights
        W_test[Y_test == 1] = (W_test[Y_test == 1] / W_test[Y_test == 1].sum()) * W_test.shape[0] / 2
        W_test[Y_test == 0] = (W_test[Y_test == 0] / W_test[Y_test == 0].sum()) * W_test.shape[0] / 2

        # Calculate ROC
        auc_score = roc_auc_score(y_true=Y_test, y_score=y_scores, sample_weight=W_test)
        
        return auc_score
    
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.8027792189265726


#### default.qubit.autograd Device

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits) 
dev =qml.device('default.qubit.autograd', wires=n_features) 
 
# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 1000, 0)

Epoch:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 260.7758
Epoch: 1, Loss: 260.4985
Epoch: 2, Loss: 259.9815
Epoch: 3, Loss: 259.1910
Epoch: 4, Loss: 258.1202
Epoch: 5, Loss: 256.7669
Epoch: 6, Loss: 255.1331
Epoch: 7, Loss: 253.2255
Epoch: 8, Loss: 251.0544
Epoch: 9, Loss: 248.6347
Epoch: 9, Validation Loss: 253.2934, AUC Score: 0.1791
Epoch: 10, Loss: 245.9862
Epoch: 11, Loss: 243.1317
Epoch: 12, Loss: 240.0952
Epoch: 13, Loss: 236.8960
Epoch: 14, Loss: 233.5446
Epoch: 15, Loss: 230.0417
Epoch: 16, Loss: 226.3849
Epoch: 17, Loss: 222.5731
Epoch: 18, Loss: 218.6095
Epoch: 19, Loss: 214.5019
Epoch: 19, Validation Loss: 215.1435, AUC Score: 0.1793
Epoch: 20, Loss: 210.2610
Epoch: 21, Loss: 205.8995
Epoch: 22, Loss: 201.4313
Epoch: 23, Loss: 196.8725
Epoch: 24, Loss: 192.2420
Epoch: 25, Loss: 187.5622
Epoch: 26, Loss: 182.8589
Epoch: 27, Loss: 178.1607
Epoch: 28, Loss: 173.4973
Epoch: 29, Loss: 168.8985
Epoch: 29, Validation Loss: 166.6355, AUC Score: 0.1789
Epoch: 30, Loss: 164.3935
Epoch: 31, Loss: 160.0106
Epoch: 32, 

In [ ]:
print (best_score)

0.8127164212937147


In [ ]:
# test the model

def test(n_features, n_layers,x_test,y_test,w_test, weights):
        # Remove grad
        X_test = np.array(x_test, requires_grad=False)
        Y_test = np.array(y_test, requires_grad=False)
        W_test = np.array(w_test, requires_grad=False)

        # This will be between -1 and 1, we need to convert to between 0 and 1
        y_scores = np.array([classifier(n_features, n_layers,weights, x) for x in X_test])
        y_scores = (y_scores + 1) / 2

        # Renormalize weights
        W_test[Y_test == 1] = (W_test[Y_test == 1] / W_test[Y_test == 1].sum()) * W_test.shape[0] / 2
        W_test[Y_test == 0] = (W_test[Y_test == 0] / W_test[Y_test == 0].sum()) * W_test.shape[0] / 2

        # Calculate ROC
        auc_score = roc_auc_score(y_true=Y_test, y_score=y_scores, sample_weight=W_test)
        
        return auc_score
    
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7030216574917096


#### qiskit.ibmq Device

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
provider = IBMProvider()
dev = ibm_dev = qml.device('qiskit.ibmq', wires=n_features, backend='ibmq_qasm_simulator', provider=provider) 

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 1000, 0)

Epoch:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 241.2400
Epoch: 1, Loss: 241.0099
Epoch: 2, Loss: 240.5597
Epoch: 3, Loss: 239.8305
Epoch: 4, Loss: 238.8123
Epoch: 5, Loss: 237.4988
Epoch: 6, Loss: 235.8874
Epoch: 7, Loss: 233.9794
Epoch: 8, Loss: 231.7807
Epoch: 9, Loss: 229.3020
Epoch: 9, Validation Loss: 232.6091, AUC Score: 0.1642
Epoch: 10, Loss: 226.5588
Epoch: 11, Loss: 223.5714
Epoch: 12, Loss: 220.3651
Epoch: 13, Loss: 216.9686
Epoch: 14, Loss: 213.4123
Epoch: 15, Loss: 209.7246
Epoch: 16, Loss: 205.9293
Epoch: 17, Loss: 202.0455
Epoch: 18, Loss: 198.0899
Epoch: 19, Loss: 194.0783
Epoch: 19, Validation Loss: 195.4333, AUC Score: 0.1642
Epoch: 20, Loss: 190.0276
Epoch: 21, Loss: 185.9571
Epoch: 22, Loss: 181.8898
Epoch: 23, Loss: 177.8518
Epoch: 24, Loss: 173.8700
Epoch: 25, Loss: 169.9698
Epoch: 26, Loss: 166.1737
Epoch: 27, Loss: 162.5018
Epoch: 28, Loss: 158.9704
Epoch: 29, Loss: 155.5921
Epoch: 29, Validation Loss: 155.8933, AUC Score: 0.1639
Epoch: 30, Loss: 152.3754
Epoch: 31, Loss: 149.3251
Epoch: 32, 

In [ ]:
print (best_score)

0.8354234198036303


In [ ]:
# test the model

def test(n_features, n_layers,x_test,y_test,w_test, weights):
        # Remove grad
        X_test = np.array(x_test, requires_grad=False)
        Y_test = np.array(y_test, requires_grad=False)
        W_test = np.array(w_test, requires_grad=False)

        # This will be between -1 and 1, we need to convert to between 0 and 1
        y_scores = np.array([classifier(n_features, n_layers,weights, x) for x in X_test])
        y_scores = (y_scores + 1) / 2

        # Renormalize weights
        W_test[Y_test == 1] = (W_test[Y_test == 1] / W_test[Y_test == 1].sum()) * W_test.shape[0] / 2
        W_test[Y_test == 0] = (W_test[Y_test == 0] / W_test[Y_test == 0].sum()) * W_test.shape[0] / 2

        # Calculate ROC
        auc_score = roc_auc_score(y_true=Y_test, y_score=y_scores, sample_weight=W_test)
        
        return auc_score
    
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.8640271361518534
